
200 examples in the public dataset leaves very little room for training!

Using `gpt-3.5-turbo` I created another 500 high quality examples at my expense [that I share freely here](https://www.kaggle.com/datasets/radek1/additional-train-data-for-llm-science-exam). They are what (as of now) pushes this notebook to the highest achieved score across the public notebooks (`0.723`)!

If you find the additional training examples useful, please upvote the dataset! 😊

👉 [additional train data for LLM Science Exam 🥳](https://www.kaggle.com/datasets/radek1/additional-train-data-for-llm-science-exam)

Thank you! Appreciate your help! 🙏🙏🙏

This notebook builds on a [notebook](https://www.kaggle.com/code/leonidkulyk/lb-0-709-llm-se-deberta-v3-large-i-1k-wiki) by LEONID KULYK. Among some of the changes are:

* use of a new high quality dataset for training
* modified training procedure carried out in the notebook
* general streamlining of code/training for readability

# Preprocessing the dataset

In [1]:
from typing import Optional, Union
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from dataclasses import dataclass
from transformers import AutoTokenizer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoModel

deberta_v3_large = '/kaggle/input/deberta-v3-large-hf-weights'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

We begin by loading and processing the train data.

In [2]:
df_train = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
df_train = df_train.drop(columns="id")
df_train.shape

(200, 7)

Let's add another 500 examples to the train set!

In [3]:
df_train = pd.concat([
    df_train,
    pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv'),
])
df_train.reset_index(inplace=True, drop=True)
df_train.shape

(700, 7)

Now that we have gone from 200 -> 700 train examples, let us preprocess the data and begin training.

In [4]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
index_to_option = {v: k for k,v in option_to_index.items()}

def preprocess(example):
    first_sentence = [example['prompt']] * 5
    second_sentences = [example[option] for option in 'ABCDE']
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    
    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

We first create a HuggingFace `Dataset`.

In [5]:
tokenizer = AutoTokenizer.from_pretrained(deberta_v3_large)

dataset = Dataset.from_pandas(df_train)
dataset

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Dataset({
    features: ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'],
    num_rows: 700
})

And let us now preprocess the examples for training.

In [6]:
tokenized_dataset = dataset.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_dataset

  0%|          | 0/700 [00:00<?, ?ex/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 700
})

# Training

In [7]:
training_args = TrainingArguments(
    warmup_ratio=0.8,
    learning_rate=5e-6,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    report_to='none',
    output_dir='.'
)

model = AutoModelForMultipleChoice.from_pretrained(deberta_v3_large)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    train_dataset=tokenized_dataset,
)

trainer.train()

Some weights of the model checkpoint at /kaggle/input/deberta-v3-large-hf-weights were not used when initializing DebertaV2ForMultipleChoice: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

Step,Training Loss
500,1.616200
1000,1.600700
1500,1.356700
2000,1.096100


TrainOutput(global_step=2100, training_loss=1.3949325597853888, metrics={'train_runtime': 586.2184, 'train_samples_per_second': 3.582, 'train_steps_per_second': 3.582, 'total_flos': 823233082746960.0, 'train_loss': 1.3949325597853888, 'epoch': 3.0})

# Predicting on the test set

Now that we have trained our model, let us predict on the test set.

In [8]:
test_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')
test_df['answer'] = 'A' # dummy answer that allows us to preprocess the test dataset just like we preprocessed the train dataset

tokenized_test_dataset = Dataset.from_pandas(test_df.drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E'])

  0%|          | 0/200 [00:00<?, ?ex/s]

In [9]:
test_predictions = trainer.predict(tokenized_test_dataset).predictions
test_predictions[:4]

array([[-5.919389 , -3.1346128, -5.882382 ,  1.061752 , -5.349337 ],
       [ 1.0768496,  0.6444651, -3.657922 , -4.1297135, -2.3705304],
       [ 2.539788 , -3.852208 ,  0.4875411, -5.173388 , -4.1340165],
       [-5.267994 , -4.3641067,  1.7363403, -3.7330499, -4.3782735]],
      dtype=float32)

The predictions are values output from the last layer of our neural network.

Let's obtain the predicted answer ids by sorting them from largest to the smallest.

In [10]:
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_ids[:3]

array([[3, 1, 4, 2, 0],
       [0, 1, 4, 2, 3],
       [0, 2, 1, 4, 3]])

Let us now assign a letter corresponding to each predicted id (0 -> 'A', 1 -> 'B', etc). 

In [11]:
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
predictions_as_answer_letters[:3]

array([['D', 'B', 'E', 'C', 'A'],
       ['A', 'B', 'E', 'C', 'D'],
       ['A', 'C', 'B', 'E', 'D']], dtype='<U1')

And let us now go from this representation to outputting a string with 3 highest rated answers seperated by a space.

In [12]:
predictions_as_string = test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]
predictions_as_string[:3]

['D B E', 'A B E', 'A C B']

And we are done! 🥳

Let us now output our submission.

In [13]:
submission = test_df[['id', 'prediction']]
submission.to_csv('submission.csv', index=False)

pd.read_csv('submission.csv').head()

,id,prediction
0,0,D B E
1,1,A B E
2,2,A C B
3,3,C D B
4,4,D A B


I hope you enjoyed this notebook!

If you found it useful, please upvote 👉 [the corresponding dataset with 500 new training examples](https://www.kaggle.com/datasets/radek1/additional-train-data-for-llm-science-exam) 👈

Thank you, appreciate your help! 🙏😊

Thank you for reading and happy Kaggling!